In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import os
import spacy
from spacy.util import compounding
from spacy.util import minibatch
import random
import warnings
import scipy.stats as stats

warnings.filterwarnings("ignore")



In [ ]:
#funcion para generar colores aleatorios

def generar_colores_aleatorios(n):
    colores = []
    for _ in range(n):
        color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        colores.append(color)
    return colores


In [ ]:
# Lectura de los datos
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
ss = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

# Eliminar valores nulos del archivo de entrenamiento
train.dropna(inplace=True)

Vamos a seleccionar train como conjunto de datos a analizar, para ello debe analizar la distribución de tweets en el conjunto y Dibujar un gráfico de embudo para una mejor visualización.

In [ ]:
# Contar la cantidad de tweets por cada tipo de sentimiento
sentiment_counts = train['sentiment'].value_counts()

# Crear una lista con los nombres de los sentimientos
sentiments = sentiment_counts.index.tolist()

# Crear una lista con los valores de los conteos de tweets
tweet_counts = sentiment_counts.tolist()

# Crear el gráfico de embudo
fig = plt.figure(figsize=(8, 6))
plt.bar(sentiments, tweet_counts)
plt.xlabel('Sentimiento')
plt.ylabel('Cantidad de tweets')
plt.title('Distribución de tweets por sentimiento en el conjunto de datos "train"')
plt.show()

In [ ]:
# Función para calcular la similitud de Jaccard
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# Agregar columna con la diferencia en el número de palabras
train['num_words_text'] = train['text'].apply(lambda x: len(str(x).split()))
train['num_words_selected_text'] = train['selected_text'].apply(lambda x: len(str(x).split()))
train['diff_num_words'] = train['num_words_text'] - train['num_words_selected_text']

# Agregar columna con las puntuaciones de similitud de Jaccard
train['jaccard_similarity'] = train.apply(lambda x: jaccard(str(x['text']), str(x['selected_text'])), axis=1)

# Gráfico de dispersión: Diferencia en el número de palabras vs. Puntuaciones de similitud de Jaccard
plt.figure(figsize=(8, 6))
plt.scatter(train['diff_num_words'], train['jaccard_similarity'], alpha=0.5)
plt.xlabel('Diferencia en el número de palabras')
plt.ylabel('Similitud de Jaccard')
plt.title('Relación entre la diferencia en el número de palabras y la similitud de Jaccard')
plt.show()


In [ ]:
# Calcular la curtosis y la asimetría de los tweets positivos
positivos = train[train['sentiment'] == 'positive']
curtosis_positivos = stats.kurtosis(positivos['jaccard_similarity'])
asimetria_positivos = stats.skew(positivos['jaccard_similarity'])

# Calcular la curtosis y la asimetría de los tweets negativos
negativos = train[train['sentiment'] == 'negative']
curtosis_negativos = stats.kurtosis(negativos['jaccard_similarity'])
asimetria_negativos = stats.skew(negativos['jaccard_similarity'])

# Calcular la curtosis y la asimetría de los tweets neutros
neutros = train[train['sentiment'] == 'neutral']
curtosis_neutros = stats.kurtosis(neutros['jaccard_similarity'])
asimetria_neutros = stats.skew(neutros['jaccard_similarity'])

# Imprimir las conclusiones
print("Conclusiones:")
print("a. Los tweets positivos tienen una alta curtosis: {:.2f}, y una asimetría: {:.2f}".format(curtosis_positivos, asimetria_positivos))
print("b. Los tweets negativos tienen una alta curtosis: {:.2f}, y una asimetría: {:.2f}".format(curtosis_negativos, asimetria_negativos))
print("c. Los tweets neutros tienen una baja curtosis: {:.2f}, y una asimetría: {:.2f}".format(curtosis_neutros, asimetria_neutros))

Visualización de la distribución de tweets por sentimiento:

In [ ]:
sentimientos = train['sentiment'].value_counts()
fig = px.bar(x=sentimientos.index, y=sentimientos.values, color=sentimientos.index)
fig.update_layout(title='Distribución de tweets por sentimiento')
fig.show()


Gráfico de embudo de tweets:

In [ ]:
fig = go.Figure(go.Funnelarea(
    text=train['sentiment'].value_counts().index,
    values=train['sentiment'].value_counts().values
))
fig.update_layout(title='Gráfico de embudo de tweets por sentimiento')
fig.show()

Word Cloud de los textos seleccionados:

In [ ]:
textos_seleccionados = ' '.join(train['selected_text'].dropna().values)
wordcloud = WordCloud(stopwords=stopwords.words('english')).generate(textos_seleccionados)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
